In [56]:
import numpy as np
import pandas as pd
import os
import re

In [57]:
file_list = [
    "January.csv", "February.csv", "March.csv", "April.csv", "May.csv", "June.csv",
    "July.csv", "August.csv", "September.csv", "October.csv", "November.csv", "December.csv"
]

standard_columns = [
    "sl_no",
    "heads_of_crime",
    "major_head",
    "minor_head",
    "current_year_upto",
    "prev_year_same_month",
    "prev_month",
    "current_month"
]

def clean_df(df):
    df = df.loc[:, ~df.columns.str.contains("^Unnamed|^\\s*$")]
    col_map = {}
    for col in df.columns:
        c = col.strip().lower().replace('.', '').replace('  ', ' ').replace(' ', '_').replace('-', '_')
        c = c.replace('heads_of_crime', 'heads_of_crime')
        if 'sl' in c and 'no' in c:
            c = 'sl_no'
        elif 'heads' in c and 'crime' in c:
            c = 'heads_of_crime'
        elif 'major' in c and 'head' in c:
            c = 'major_head'
        elif 'minor' in c and 'head' in c:
            c = 'minor_head'
        elif 'current_year' in c or 'current_year_upto' in c or 'month_under_review' in c:
            c = 'current_year_upto'
        elif 'corresponding_month' in c:
            c = 'prev_year_same_month'
        elif 'previous_month' in c and 'corresponding' not in c:
            c = 'prev_month'
        elif 'current_month' in c:
            c = 'current_month'
        col_map[col] = c
    df = df.rename(columns=col_map)
    for col in standard_columns:
        if col not in df.columns:
            df[col] = pd.NA
    df = df[standard_columns]
    return df

output_dir = "cleaned_csvs"
os.makedirs(output_dir, exist_ok=True)

In [58]:
dfs = []
for fname in file_list:
    try:
        try:
            df = pd.read_csv(fname, encoding="utf-8")
        except UnicodeDecodeError:
            df = pd.read_csv(fname, encoding="latin1")
        df_clean = clean_df(df)
        month_name = os.path.splitext(os.path.basename(fname))[0]
        df_clean['month'] = month_name
        dfs.append(df_clean)
        out_path = os.path.join(output_dir, fname)
        df_clean.to_csv(out_path, index=False)
    except Exception as e:
        print(f"Error processing {fname}: {e}")

final_df = pd.concat(dfs, ignore_index=True)
final_df.to_csv("all_months_concatenated.csv", index=False)

print(f"Cleaned individual files saved in '{output_dir}/'")
print("Concatenated file saved as 'all_months_concatenated.csv'")

Cleaned individual files saved in 'cleaned_csvs/'
Concatenated file saved as 'all_months_concatenated.csv'


In [59]:
final_df.shape

(8467, 9)

In [60]:
final_df.head()

,sl_no,heads_of_crime,major_head,minor_head,current_year_upto,prev_year_same_month,prev_month,current_month,month
0,1.0,A - IPC Crime,Murder (Sec.302/303 IPC),For gain,3.0,2.0,4.0,3.0,January
1,2.0,A - IPC Crime,Murder (Sec.302/303 IPC),Over Property Dispute,2.0,0.0,4.0,2.0,January
2,3.0,A - IPC Crime,Murder (Sec.302/303 IPC),Due to Personal Vendetta or enemity,1.0,1.0,2.0,1.0,January
3,4.0,A - IPC Crime,Murder (Sec.302/303 IPC),Due to Sexual jealousy,0.0,2.0,0.0,0.0,January
4,5.0,A - IPC Crime,Murder (Sec.302/303 IPC),For dowry by burning,0.0,0.0,0.0,0.0,January


In [61]:
final_df.drop('sl_no', axis=1)

,heads_of_crime,major_head,minor_head,current_year_upto,prev_year_same_month,prev_month,current_month,month
0,A - IPC Crime,Murder (Sec.302/303 IPC),For gain,3.0,2.0,4.0,3.0,January
1,A - IPC Crime,Murder (Sec.302/303 IPC),Over Property Dispute,2.0,0.0,4.0,2.0,January
2,A - IPC Crime,Murder (Sec.302/303 IPC),Due to Personal Vendetta or enemity,1.0,1.0,2.0,1.0,January
3,A - IPC Crime,Murder (Sec.302/303 IPC),Due to Sexual jealousy,0.0,2.0,0.0,0.0,January
4,A - IPC Crime,Murder (Sec.302/303 IPC),For dowry by burning,0.0,0.0,0.0,0.0,January
...,...,...,...,...,...,...,...,...
8462,NaN,Murder,NaN,99.0,8.0,4.0,7.0,December
8463,NaN,Rape,NaN,186.0,13.0,14.0,14.0,December
8464,NaN,Kidnapping,NaN,124.0,7.0,10.0,7.0,December
8465,NaN,Offences under the Protection of Civil Rights ...,NaN,0.0,0.0,0.0,0.0,December


In [62]:
final_df.drop_duplicates()
final_df.shape

(8467, 9)

In [63]:
MOTIVE_MAPPING = {
    'gain': ['gain', 'ransom', 'cheating', 'extortion', 'misappropriation', 'for money'],
    'enmity': ['enemity', 'vendetta', 'revenge', 'rivalry', 'animosity', 'feud'],
    'sexual motive': ['sexual', 'rape', 'molestation', 'eve teasing', 'pornography', 'jealousy', 'adultery', 'unnatural sex', 'lust'],
    'property dispute': ['property', 'land', 'house', 'dispute', 'boundary', 'possession', 'real estate'],
    'dowry': ['dowry'],
    'communal': ['communal', 'caste', 'religion', 'sectarian', 'casteism'],
    'personal': ['personal', 'family', 'domestic', 'relationship', 'marriage'],
    'political': ['political', 'election', 'party', 'vote'],
    'quarrel': ['quarrel', 'argument', 'fight', 'brawl', 'altercation'],
    'professional': ['professional', 'misconduct', 'negligence', 'malpractice'],
    'theft': ['theft', 'robbery', 'burglary', 'dacoity', 'steal', 'larceny'],
    'kidnapping': ['kidnapping', 'abduction', 'missing'],
    'negligence': ['negligence', 'rash', 'accident'],
    'substance': ['liquor', 'drug', 'narcotic', 'psychotropic', 'hooch'],
    'public order': ['riot', 'affray', 'unlawful assembly', 'public nuisance'],
    'cyber': ['cyber', 'it act', 'online', 'internet'],
    'state security': ['sedition', 'terrorism', 'naxalism', 'espionage', 'waging war'],
    'other': []  # Fallback category
}

def generalize_motive(minor_head):
    """Map specific minor heads to generalized motives"""
    if not isinstance(minor_head, str):
        return minor_head
        
    motive_str = minor_head.lower()
    
    # Special cases with priority handling
    if 'dowry' in motive_str:
        return 'dowry'
    if 'sexual' in motive_str or 'rape' in motive_str:
        return 'sexual motive'
    
    # Pattern-based mapping
    for general_motive, keywords in MOTIVE_MAPPING.items():
        for keyword in keywords:
            # Use word boundaries to avoid partial matches
            if re.search(rf'\b{re.escape(keyword)}\b', motive_str):
                return general_motive
    
    if 'dispute' in motive_str:
        return 'property dispute' if 'property' in motive_str else 'personal'
    if 'quarrel' in motive_str:
        return 'quarrel'
    
    return minor_head  # Return original if no match

final_df['general_motive'] = final_df['minor_head'].apply(generalize_motive)
final_df.to_csv('crime_data_with_general_motives.csv', index=False)

In [64]:
final_df.head()

,sl_no,heads_of_crime,major_head,minor_head,current_year_upto,prev_year_same_month,prev_month,current_month,month,general_motive
0,1.0,A - IPC Crime,Murder (Sec.302/303 IPC),For gain,3.0,2.0,4.0,3.0,January,gain
1,2.0,A - IPC Crime,Murder (Sec.302/303 IPC),Over Property Dispute,2.0,0.0,4.0,2.0,January,property dispute
2,3.0,A - IPC Crime,Murder (Sec.302/303 IPC),Due to Personal Vendetta or enemity,1.0,1.0,2.0,1.0,January,enmity
3,4.0,A - IPC Crime,Murder (Sec.302/303 IPC),Due to Sexual jealousy,0.0,2.0,0.0,0.0,January,sexual motive
4,5.0,A - IPC Crime,Murder (Sec.302/303 IPC),For dowry by burning,0.0,0.0,0.0,0.0,January,dowry
